In [25]:
import backtrader as bt
import pandas as pd

data = bt.feeds.GenericCSVData(
    dataname="../data/BTCUSDT_FIVE2025_15m.csv",
    dtformat='%Y-%m-%d %H:%M:%S',
    timeframe=bt.TimeFrame.Minutes,
    compression=15,
    datetime=0,
    open=1,
    high=2,
    low=3,
    close=4,
    volume=5,
    openinterest=-1  # No open interest in your CSV
)


class EmaCrossover(bt.Strategy):
    params = (
        ('short_period', 9),
        ('long_period', 21),
        ('position_pct', 0.95),
    )

    def __init__(self):
        self.short_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.short_period)
        self.long_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.long_period)

        self.crossover = bt.indicators.CrossOver(self.short_ema, self.long_ema)

          # Initialize DataFrame with proper dtypes
        self.trade_pairs = pd.DataFrame({
            'trade_number': pd.Series(dtype='int'),
            'entry_type': pd.Series(dtype='str'),
            'entry_price': pd.Series(dtype='float'),
            'entry_datetime': pd.Series(dtype='datetime64[ns]'),
            'exit_type': pd.Series(dtype='str'),
            'exit_price': pd.Series(dtype='float'),
            'exit_datetime': pd.Series(dtype='datetime64[ns]'),
            'size': pd.Series(dtype='float'),
            'pnl': pd.Series(dtype='float'),
            'pnl_percentage': pd.Series(dtype='float') 
        })
        
        self.trade_counter = 0  # To track trade pair numbers
        self.current_trade = None  # To track current open trade

        # Simple counters
        self.total_trades = 0
        self.winning_trades = 0
        self.total_pnl = 0.0

        self.order= None  # To keep track of pending orders

    def next(self):
        if len(self) < self.params.long_period:
            return
        
        # Skip if we have pending orders
        if self.order:
            return
        
        # Get current position
        position = self.position.size

        if self.crossover < 0:
            # print(f"🔥BEARISH CROSSOVER - Going SHORT")
             # Calculate how many units you can afford
            cash = self.broker.getcash()
            price = self.data.close[0]
            size = (cash * self.params.position_pct) / price
            
            # print(f"Cash: ${cash:.2f}, Price: ${price:.2f}, Size: {size}")
            self.order = self.sell(size=size)
        
        elif self.crossover > 0 and position < 0:
            # print(f"🔥 BULLISH CROSSOVER - Closing SHORT")

            # Close the entire short position
            # print(
                # f"Current short position: {position} ---- Price: ${self.data.close[0]:.2f}")
            self.order = self.buy(size=abs(position))


    def stop(self):
        position = self.position.size
        
        if position != 0:
            print(f"🔚 END OF DATA - Force closing position: {position} units")
            
            # If we have an open trade, close it
            if self.current_trade:
                current_datetime = self.data.datetime.datetime(0)
                current_price = self.data.close[0]
                
                if position > 0:
                    # Closing LONG position
                    self.current_trade['exit_type'] = 'SELL'
                    self.current_trade['exit_price'] = current_price
                    self.current_trade['exit_datetime'] = current_datetime
                    
                    # Calculate PnL for LONG trade
                    pnl = (current_price - self.current_trade['entry_price']) * self.current_trade['size']
                    self.current_trade['pnl'] = pnl

                    # pnl = (current_price - self.current_trade['entry_price']) * self.current_trade['size']
                    # self.current_trade['pnl'] = pnl
                    pnl_percentage = ((current_price - self.current_trade['entry_price']) / self.current_trade['entry_price']) * 100
                    self.current_trade['pnl_percentage'] = pnl_percentage
                    
                else:
                    # Closing SHORT position
                    self.current_trade['exit_type'] = 'BUY'
                    self.current_trade['exit_price'] = current_price
                    self.current_trade['exit_datetime'] = current_datetime
                    
                    # Calculate PnL for SHORT trade
                    pnl = (self.current_trade['entry_price'] - current_price) * self.current_trade['size']
                    self.current_trade['pnl'] = pnl
                    pnl_percentage = ((self.current_trade['entry_price'] - current_price) / self.current_trade['entry_price']) * 100
                    self.current_trade['pnl_percentage'] = pnl_percentage
                
                # Add the forced-closed trade to DataFrame
                self.trade_pairs = pd.concat([self.trade_pairs, pd.DataFrame([self.current_trade])], ignore_index=True)
                self.current_trade = None
            
            # Your existing closing logic...
            if position > 0:
                self.sell(size=abs(position))
            else:
                self.buy(size=abs(position))
            print("=" * 50)
   
    def notify_order(self, order):
        if order.status == order.Completed:
            current_datetime = self.data.datetime.datetime(0)
            
            if order.isbuy():
                # print(f"BUY EXECUTED at ${order.executed.price:.2f} , position size: {order.executed.size}")
                
                # If this is closing a SHORT position (buying to close)
                if self.current_trade and self.current_trade['entry_type'] == 'SELL':
                    # This is an EXIT - complete the trade pair
                    self.current_trade['exit_type'] = 'BUY'
                    self.current_trade['exit_price'] = order.executed.price
                    self.current_trade['exit_datetime'] = current_datetime
                    
                    # Calculate PnL for SHORT trade: (entry_price - exit_price) * size
                    pnl = (self.current_trade['entry_price'] - order.executed.price) * self.current_trade['size']
                    self.current_trade['pnl'] = pnl
                    pnl_percentage = ((self.current_trade['entry_price'] - order.executed.price) / self.current_trade['entry_price']) * 100
                    self.current_trade['pnl_percentage'] = pnl_percentage
                    
                    # Add completed trade to DataFrame
                    self.trade_pairs = pd.concat([self.trade_pairs, pd.DataFrame([self.current_trade])], ignore_index=True)
                    self.current_trade = None  # Reset current trade
                    
                else:
                    # This is an ENTRY for a LONG position
                    self.trade_counter += 1
                    self.current_trade = {
                        'trade_number': self.trade_counter,
                        'entry_type': 'BUY',
                        'entry_price': order.executed.price,
                        'entry_datetime': current_datetime,
                        'exit_type': None,
                        'exit_price': None,
                        'exit_datetime': None,
                        'size': abs(order.executed.size),
                        'pnl': None,
                        'pnl_percentage': None
                    }
                    
            elif order.issell():
                # print(f"SELL EXECUTED at ${order.executed.price:.2f} , position size: {order.executed.size}")
                
                # If this is closing a LONG position (selling to close)
                if self.current_trade and self.current_trade['entry_type'] == 'BUY':
                    # This is an EXIT - complete the trade pair
                    self.current_trade['exit_type'] = 'SELL'
                    self.current_trade['exit_price'] = order.executed.price
                    self.current_trade['exit_datetime'] = current_datetime
                    
                    # Calculate PnL for LONG trade: (exit_price - entry_price) * size
                    pnl = (order.executed.price - self.current_trade['entry_price']) * self.current_trade['size']
                    self.current_trade['pnl'] = pnl
                    pnl_percentage = ((order.executed.price - self.current_trade['entry_price']) / self.current_trade['entry_price']) * 100
                    self.current_trade['pnl_percentage'] = pnl_percentage


                    # Add completed trade to DataFrame
                    self.trade_pairs = pd.concat([self.trade_pairs, pd.DataFrame([self.current_trade])], ignore_index=True)
                    self.current_trade = None  # Reset current trade
                    
                else:
                    # This is an ENTRY for a SHORT position
                    self.trade_counter += 1
                    self.current_trade = {
                        'trade_number': self.trade_counter,
                        'entry_type': 'SELL',
                        'entry_price': order.executed.price,
                        'entry_datetime': current_datetime,
                        'exit_type': None,
                        'exit_price': None,
                        'exit_datetime': None,
                        'size': abs(order.executed.size),
                        'pnl': None,
                        'pnl_percentage': None
                    }
        
        # Your existing error handling code...
        elif order.status == order.Margin:
            print(f"   ❌ ORDER REJECTED - Insufficient Margin/Cash")
        elif order.status == order.Rejected:
            print(f"   ❌ ORDER REJECTED - {order.info}")
        elif order.status == order.Canceled:
            print(f"   ⚠️ ORDER CANCELED")
            
        # Reset self.order when order completes
        if order.status in [order.Completed, order.Canceled, order.Rejected, order.Margin]:
            self.order = None


    # def notify_trade(self, trade):
    #     """Handle trade notifications"""
    #     if trade.isclosed:
    #         self.total_trades += 1
    #         pnl = trade.pnl
    #         self.total_pnl += pnl

    #         # Determine trade direction
    #         direction = "LONG" if trade.size > 0 else "SHORT"

    #         if pnl > 0:
    #             self.winning_trades += 1
    #             print(f"   ✅ WINNING {direction} TRADE - P&L: ${pnl:.2f}")
    #         else:
    #             print(f"   ❌ LOSING {direction} TRADE - P&L: ${pnl:.2f}")

    #         # Print running statistics
    #         win_rate = (self.winning_trades / self.total_trades) * 100
    #         print(
    #             f"   📊 Total: {self.total_trades} | Win Rate: {win_rate:.1f}% | Total P&L: ${self.total_pnl:.2f}")
    #         print("=" * 50)

  

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(EmaCrossover)


cerebro.broker.setcash(10000.0)
cerebro.broker.setcommission(commission=0.001)  # 0.1% commission


# Run the strategy and capture the strategy instance
results = cerebro.run()
strategy_instance = results[0]

# Access the enhanced trade pairs DataFrame
trade_pairs_df = strategy_instance.trade_pairs

print("\n📊 COMPLETE TRADE PAIRS:")
# print(trade_pairs_df)
trade_pairs_df

# print("=" * 50)
print(f"🏁 Final Portfolio Value: ${cerebro.broker.getvalue():.2f}")

🔚 END OF DATA - Force closing position: -0.051336496455438446 units

📊 COMPLETE TRADE PAIRS:
🏁 Final Portfolio Value: $5741.36
